In [ ]:
from pyspark.sql import SparkSession

## SPARK SESSION

In [ ]:
spark = SparkSession.builder.appName("sparkhive").enableHiveSupport().getOrCreate()

## Data Extraction from Hive

In [ ]:
df = spark.sql("SELECT * FROM fraud_project.fraud_full_load_external")
df.show()

## Exploratory Data Analysis (EDA)

#Statistical Analysis
#Plot some feature distributions
#Correlation Matrix

## Feature Engineering

#Null Values
#Imputation Techniques 
#Scale the data
#One Hot Encoding
#Techniques to balance the data (Oversampling and Undersampling Techniques)

## MODELING

#Split the dataset into training and testing
#Logistic regression, 2 Ensemble techniques (Randon Forest and XGBoost)


## EVALUATION

#Accuracy
#Recall
#Precision
#F1 Score
#AUC-ROC